# STAT3009 Project 1
- LAW Yiu Leung Eric 1155149315

## Load the developed methods

In [1]:
import numpy as np
from developed_methods import *

## Pre-process the raw data
- check the `user_id` and `item_id`: mapping `item_id` to a continuous sequence based on `sklean.preprocessing`
- use `sklearn.model_selection.train_test_split` to generate `train` and `test` dataset
- create `train_pair`,`train_rating`, `test_pair`

In [2]:
import pandas as pd
df = pd.read_csv("data/train.csv")
dtest_submit = pd.read_csv("data/test.csv")

## mapping 
from sklearn import preprocessing
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(df['user_id'], dtest_submit["user_id"]))
df['user_id'] = le_user.transform(df["user_id"])
dtest_submit["user_id"] = le_user.transform(dtest_submit["user_id"])

le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(df['item_id'], dtest_submit["item_id"]))
df["item_id"] = le_item.transform(df["item_id"])
dtest_submit["item_id"] = le_item.transform(dtest_submit["item_id"])

## generate train / test dataset
from sklearn.model_selection import train_test_split
dtrain, dtest = train_test_split(df, test_size=0.33, random_state=42)

## save real ratings for test set for evaluation.
test_rating = np.array(dtest['rating'])

## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')

Create train_pair, train_rating and test_pair

In [3]:
# train_pair, train_rating
train_pair = dtrain[['user_id', 'item_id']].values
train_rating = dtrain['rating'].values

# test_pair
test_pair = dtest[['user_id', 'item_id']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

# Impelement deep learning with Keras

In [4]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

## Create the model
We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias.

In [5]:
class LFactorNet(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(LFactorNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.glb_bias = tf.Variable(0., trainable=True) 
        self.movie_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.movie_bias = layers.Embedding(num_items, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias + self.glb_bias
        return x

In [7]:
model = LFactorNet(num_users=n_user, num_items=n_item, embedding_size=50)


metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=train_pair,
    y=train_rating,
    batch_size=64,
    epochs=200,
    verbose=1,
    validation_split=.2,
)

Epoch 1/200
129/129 [==============================] - 2s 4ms/step - loss: 46.4377 - mae: 5.9697 - rmse: 6.7121 - val_loss: 43.8266 - val_mae: 5.7573 - val_rmse: 6.5201
Epoch 2/200
129/129 [==============================] - 0s 2ms/step - loss: 33.6719 - mae: 4.8097 - rmse: 5.6781 - val_loss: 40.0347 - val_mae: 5.4005 - val_rmse: 6.2015
Epoch 3/200
129/129 [==============================] - 0s 2ms/step - loss: 21.8221 - mae: 3.4677 - rmse: 4.4763 - val_loss: 33.2343 - val_mae: 4.7102 - val_rmse: 5.5875
Epoch 4/200
129/129 [==============================] - 0s 2ms/step - loss: 13.6518 - mae: 2.4192 - rmse: 3.3856 - val_loss: 26.7687 - val_mae: 3.9902 - val_rmse: 4.9420
Epoch 5/200
129/129 [==============================] - 0s 2ms/step - loss: 11.6849 - mae: 2.2524 - rmse: 3.0519 - val_loss: 24.6538 - val_mae: 3.7524 - val_rmse: 4.7199
Epoch 6/200
129/129 [==============================] - 0s 2ms/step - loss: 11.4125 - mae: 2.2476 - rmse: 3.0117 - val_loss: 24.0462 - val_mae: 3.6983 - val

In [8]:
## make prediction
pred_rating = model.predict(test_pair).flatten()
print(pred_rating)
print('rmse: LFactorNet: %.3f' %np.sqrt(np.mean((pred_rating - test_rating)**2)))

[6.090626  5.8424387 4.087761  ... 3.3490307 6.1221333 3.8314614]
rmse: LFactorNet: 1.400


In [9]:
pred_rating = model.predict(dtest_submit[["user_id", "item_id"]]).flatten()
print(pred_rating)

[2.6130528 4.703871  5.0403795 ... 3.0259562 8.935659  4.4722023]


In [ ]:
submit = pd.DataFrame({"Id": range(len(dtest_submit)), "rating": pred_rating})
submit.to_csv("predict/submission1.csv", index = False)